In [10]:
import pandas as pd
import numpy as np
import scipy



from scipy.stats import ttest_1samp



In [12]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not sshow tyis message')

In [21]:
data = pd.read_excel('AB_Test_Results.xlsx')
data.info()
data.shape
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [14]:
data.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [16]:
### Просмотр данных о пропущенных значениях
data.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [17]:
### Описательная статистика для типов данных
data.groupby('VARIANT_NAME').describe()

USER_ID                                                           
               count         mean          std  min     25%     50%      75%   
VARIANT_NAME                                                                   
control       4984.0  4989.436798  2905.145109  2.0  2466.0  4964.5  7576.25  \
variant       5016.0  4972.776914  2876.320625  3.0  2476.5  4958.5  7415.25   

                      REVENUE                                                  
                  max   count      mean       std  min  25%  50%  75%     max  
VARIANT_NAME                                                                   
control       10000.0  4984.0  0.129013  3.007524  0.0  0.0  0.0  0.0  196.01  
variant       10000.0  5016.0  0.070070  1.314802  0.0  0.0  0.0  0.0   58.63

In [19]:
### Создание таблицы с повторяющимися USER_ID в выборках
Double_ID = data.\
    groupby("USER_ID", as_index = False). \
    agg({"VARIANT_NAME": data.Series.nunique})
print(Double_ID)

AttributeError: 'DataFrame' object has no attribute 'Series'

In [22]:
### кол-во по группам
data.VARIANT_NAME.value_counts()

VARIANT_NAME
variant    5016
control    4984
Name: count, dtype: int64